In [ ]:
#import libraries
import os
import pandas as pd
import re
import csv
from io import StringIO
from pathlib import Path

In [ ]:
#read file
filename = Path(input("Enter the file name : "))

In [ ]:
#load csv to dataframe
col_list = ['post_date', 'sale_date', 'desc', 'amount']
df = pd.read_csv(filename, names = col_list)

In [ ]:
#drop unneeded columns
df.drop(['post_date'], axis = 1, inplace = True)

In [ ]:
#fix date
#extract day month year
from datetime import datetime

day = df.sale_date.str.extract(r'(\d{2})', expand = False)
month = df.sale_date.str.extract(r'([A-Za-z]{3})', expand = False)
year = str(datetime.now().year)

# refill column
df['date'] = pd.to_datetime((month + ' ' + day + ', ' + year)).dt.strftime("%m/%d/%Y")


In [ ]:
#extract text from sale_date
df['desc01'] = df.sale_date.str.replace('\d{2} [A-Za-z]{3}','').str.strip()

In [ ]:
#get rid of nan rows
df = df.loc[-df.date.isna()]

In [ ]:
#fix desc
df['desc'] = df.desc.str.replace('\n',' ')

In [ ]:
df.desc = df.desc.fillna('')

In [ ]:
#combine descriptions
df['desc'] = df.desc01 + ' ' + df.desc

In [ ]:
#create deposit and withdraw columns
df['debit'] = df['amount'].apply(lambda x: x if ('CR'not in x) else '0')
df['credit'] = df['amount'].apply(lambda x: x if ('CR' in x) else '0').str.replace('CR','').fillna('0')

In [ ]:
#drop unneeded columns
df.drop(['sale_date','amount', 'desc01'], axis = 1, inplace = True)

In [ ]:
#set order
df = df[['date','desc','debit','credit']]

In [ ]:
#strip unneeded spaces
df.desc = df.desc.str.strip()
df.desc = df.desc.str.replace(" +"," ")

In [ ]:
#lower
df['desc'] = df.desc.str.lower()

In [ ]:
#replace spaces with decimal in dirty amounts
#(df.debit.str.isnumeric())
df.debit = df.debit.str.replace(' ','.')

In [ ]:
#export to csv
filename_csv_full = 'for import ' + filename.name
export_csv = df.to_csv (filename_csv_full, index = None, header=True)